## Общие методы для обработки данных
В данном разделе будут находится общие методы.

In [97]:
import pandas as pd

# читаем данные и создаём таблицу
def data_loader(path: str):
    return pd.read_csv(path)

# преобразуем данные так, как нам необходимо
def data_type_and_date_transform_alter(data: pd.DataFrame) ->pd.DataFrame:
    data = data.replace(",", "", regex=True)
    data.Date = pd.to_datetime(data.Date)
    if 'Ticker' in data:
        data = data.drop(columns = ['Ticker'])
    if 'Per' in data:
        data = data.drop(columns = ['Per'])
    if 'Date.1' in data:
        data = data.drop(columns = ['Date.1'])
    data = data.sort_values(by=["Date"])
    data.set_index('Date')
    if "Price" in data:
        data = data.rename(columns={'Price': 'Close'})
    data.Close = data.Close.astype(float)
    data.High = data.High.astype(float)
    data.Low = data.Low.astype(float)
    data.Open = data.Open.astype(float)
    if 'Vol' in data:
        data["Vol"] = data["Vol"].astype(float)
    return data

def data_type_and_date_transform(data: pd.DataFrame) ->pd.DataFrame:
    data = data.replace(",", "", regex=True)
    data.Date = pd.to_datetime(data.Date)
    data = data.sort_values(by=["Date"])
    data.set_index('Date')
    data.Price = data.Price.astype(float)
    data.High = data.High.astype(float)
    data.Low = data.Low.astype(float)
    data.Open = data.Open.astype(float)
    data["Vol."] = data["Vol."].astype(float)
    return data

# Обрезает данные по дате. 
# Важно, чтобы даты в датафрейме были в том же формате что и начальная и конечная дата
def cut_by_date(data: pd.DataFrame, begin_date: str, end_date: str) -> pd.DataFrame:
     result = data[(data['Date'] > begin_date) & (data['Date'] < end_date)]
     result.index = data.index[:len(result)]
     return result

## Смещение данных с запоминанием их реального положения
Поскольку нам необходимы данные без пропущенных дней, то один из вариантов это смещение данных с запоминанием реальной даты. В данном случае все дни будут идти без пропусков.

In [2]:
# Считываем данные
general_data = data_loader("../../Data/Day/S&P 500 Historical Data00-20.csv")
general_data = data_type_and_date_transform(general_data)

In [3]:
# Необходимо развернуть данные и исправить индексы
general_data.index =general_data.index[::-1]

In [4]:
# Проверяем что с данными всё ок
general_data.head()

,Date,Price,Open,High,Low,Vol.,Change %
0,2000-01-03,1455.2,1469.2,1478.0,1438.4,NaN,-0.95%
1,2000-01-04,1399.4,1455.2,1455.2,1397.4,NaN,-3.83%
2,2000-01-05,1402.1,1399.4,1413.3,1377.7,NaN,0.19%
3,2000-01-06,1403.5,1402.1,1411.9,1392.0,NaN,0.10%
4,2000-01-07,1441.5,1403.5,1441.5,1400.5,NaN,2.71%


In [5]:
# Производим обрезку данных 
special_data = cut_by_date(general_data, "2010-01-01", "2014-01-01")

In [6]:
# Проверяем получилось ли сделать обрезку
special_data

,Date,Price,Open,High,Low,Vol.,Change %
0,2010-01-04,1132.99,1116.56,1133.87,1116.56,NaN,1.60%
1,2010-01-05,1136.52,1132.66,1136.63,1129.66,NaN,0.31%
2,2010-01-06,1137.14,1135.71,1139.19,1133.95,NaN,0.05%
3,2010-01-07,1141.69,1136.27,1142.46,1131.32,NaN,0.40%
4,2010-01-08,1144.98,1140.52,1145.39,1136.22,NaN,0.29%
...,...,...,...,...,...,...,...
1001,2013-12-24,1833.32,1828.02,1833.32,1828.02,NaN,0.29%
1002,2013-12-26,1842.02,1834.96,1842.84,1834.96,NaN,0.47%
1003,2013-12-27,1841.40,1842.97,1844.89,1839.81,NaN,-0.03%
1004,2013-12-30,1841.07,1841.47,1842.47,1838.77,NaN,-0.02%


In [7]:
special_data['new_index'] = special_data.Price
special_data['new_date'] = special_data.Price
special_data.head()

C:\Users\Skiv2\AppData\Local\Temp\ipykernel_40388\2126971332.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  special_data['new_index'] = special_data.Price
C:\Users\Skiv2\AppData\Local\Temp\ipykernel_40388\2126971332.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  special_data['new_date'] = special_data.Price


,Date,Price,Open,High,Low,Vol.,Change %,new_index,new_date
0,2010-01-04,1132.99,1116.56,1133.87,1116.56,NaN,1.60%,1132.99,1132.99
1,2010-01-05,1136.52,1132.66,1136.63,1129.66,NaN,0.31%,1136.52,1136.52
2,2010-01-06,1137.14,1135.71,1139.19,1133.95,NaN,0.05%,1137.14,1137.14
3,2010-01-07,1141.69,1136.27,1142.46,1131.32,NaN,0.40%,1141.69,1141.69
4,2010-01-08,1144.98,1140.52,1145.39,1136.22,NaN,0.29%,1144.98,1144.98


In [8]:
import datetime
import calendar
import time
# Получаем значения нчального дня(в тестовом варианте это значение получается равным 1262563200)
special_data.loc[0, 'new_index'] = calendar.timegm(time.strptime(str(special_data.Date[0])[0:10], '%Y-%m-%d')) 
special_data.loc[0, 'new_date'] = datetime.datetime.fromtimestamp(special_data.loc[0, 'new_index']).strftime('%Y-%m-%d %H:%M:%S')
for i in range(len(special_data.new_index)):
    if i > 0:
        special_data.loc[i, 'new_index'] = special_data.new_index[i - 1] + 86400
        special_data.loc[i, 'new_date'] = datetime.datetime.fromtimestamp(special_data.loc[i, 'new_index']).strftime('%Y-%m-%d %H:%M:%S')

C:\Users\Skiv2\AppData\Local\Temp\ipykernel_40388\2714941934.py:6: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '2010-01-04 07:00:00' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  special_data.loc[0, 'new_date'] = datetime.datetime.fromtimestamp(special_data.loc[0, 'new_index']).strftime('%Y-%m-%d %H:%M:%S')


In [9]:
#change indexes
special_data.set_index('new_date', inplace=True)

In [10]:
# Смотрим результат
special_data.head(20)

,Date,Price,Open,High,Low,Vol.,Change %,new_index
new_date,,,,,,,,
2010-01-04 07:00:00,2010-01-04,1132.99,1116.56,1133.87,1116.56,NaN,1.60%,1.262563e+09
2010-01-05 07:00:00,2010-01-05,1136.52,1132.66,1136.63,1129.66,NaN,0.31%,1.262650e+09
2010-01-06 07:00:00,2010-01-06,1137.14,1135.71,1139.19,1133.95,NaN,0.05%,1.262736e+09
2010-01-07 07:00:00,2010-01-07,1141.69,1136.27,1142.46,1131.32,NaN,0.40%,1.262822e+09
2010-01-08 07:00:00,2010-01-08,1144.98,1140.52,1145.39,1136.22,NaN,0.29%,1.262909e+09
2010-01-09 07:00:00,2010-01-11,1146.98,1145.96,1149.74,1142.02,NaN,0.17%,1.262995e+09
2010-01-10 07:00:00,2010-01-12,1136.22,1143.81,1143.81,1131.77,NaN,-0.94%,1.263082e+09
2010-01-11 07:00:00,2010-01-13,1145.68,1137.31,1148.40,1133.18,NaN,0.83%,1.263168e+09
2010-01-12 07:00:00,2010-01-14,1148.46,1145.68,1150.41,1143.80,NaN,0.24%,1.263254e+09


## Добавление в выходные дни цены предыдущего дня
Другой вариант создания данных без промежутков. В данном случае при отсутствие следующего убирается пустой промежуток по формуле day_i = (day_prev_work + day_next_work) /2

In [116]:
# Считываем данные
general_data = data_loader("../../Data/Day/dje_Linear.csv")
general_data = data_type_and_date_transform_alter(general_data)

In [117]:
# Необходимо развернуть данные и исправить индексы
general_data.index = general_data.index[::-1]

In [118]:
# Проверяем что с данными всё ок
general_data.head()

,Unnamed: 0,Date,Open,High,Low,Close,Vol
1457,0,2010-01-04,10580.78,10583.13,10580.78,10583.13,0.0
1456,1,2010-01-05,10568.24,10568.92,10567.41,10568.92,0.0
1455,2,2010-01-06,10571.87,10573.00,10571.79,10572.47,0.0
1454,3,2010-01-07,10605.12,10608.07,10605.12,10608.07,0.0
1453,4,2010-01-08,10612.00,10614.19,10611.92,10614.19,0.0


In [119]:
# Производим обрезку данных 
special_data = cut_by_date(general_data, "2010-01-01", "2014-01-01")

In [120]:
# Проверяем получилось ли сделать обрезку
special_data.head(30)

,Unnamed: 0,Date,Open,High,Low,Close,Vol
1457,0,2010-01-04,10580.78,10583.130000,10580.780000,10583.130,0.0
1456,1,2010-01-05,10568.24,10568.920000,10567.410000,10568.920,0.0
1455,2,2010-01-06,10571.87,10573.000000,10571.790000,10572.470,0.0
1454,3,2010-01-07,10605.12,10608.070000,10605.120000,10608.070,0.0
1453,4,2010-01-08,10612.00,10614.190000,10611.920000,10614.190,0.0
1452,5,2010-01-09,10617.66,10618.190000,10617.660000,10618.190,0.0
1451,6,2010-01-10,10639.77,10641.205000,10639.655000,10641.205,0.0
1450,7,2010-01-11,10661.88,10664.220000,10661.650000,10664.220,0.0
1449,8,2010-01-12,10620.91,10621.590000,10620.540000,10621.440,0.0
1448,9,2010-01-13,10688.10,10688.100000,10684.470000,10684.470,0.0


In [121]:
import pandas as pd

# Преобразуем столбец 'Date' в формат datetime
special_data['Date'] = pd.to_datetime(special_data['Date'])

# Устанавливаем 'Date' как индекс DataFrame
special_data.set_index('Date', inplace=True)

# Ресемплируем DataFrame по дням и интерполируем пропущенные значения
df_resampled = special_data.resample('W').interpolate(method='linear')

# Сбрасываем индекс, чтобы 'Date' снова стал столбцом
df_resampled.reset_index(inplace=True)

In [122]:
df_resampled.head(20)

,Date,Unnamed: 0,Open,High,Low,Close,Vol
0,2010-01-10,6.0,10639.770000,10641.205000,10639.655000,10641.205000,0.0
1,2010-01-17,13.0,10648.420000,10648.446667,10648.066667,10648.120000,0.0
2,2010-01-24,20.0,10187.300000,10188.435000,10187.040000,10187.190000,0.0
3,2010-01-31,27.0,10128.390000,10128.505000,10127.070000,10127.185000,0.0
4,2010-02-07,34.0,9964.395000,9964.545000,9962.620000,9963.485000,0.0
5,2010-02-14,41.0,10152.853333,10155.146667,10150.913333,10153.333333,0.0
6,2010-02-21,48.0,10388.485000,10390.300000,10388.410000,10390.185000,0.0
7,2010-02-28,55.0,10363.845000,10364.640000,10363.695000,10364.185000,0.0
8,2010-03-07,62.0,10558.560000,10559.170000,10558.260000,10558.525000,0.0
9,2010-03-14,69.0,10623.595000,10623.745000,10620.570000,10622.650000,0.0


In [123]:
if "Unnamed: 0" in df_resampled:
    df_resampled = df_resampled.drop(columns = ["Unnamed: 0"])

In [124]:
df_resampled.to_csv("dje_Linear.csv")